- collections：数据集
- 拷贝：不修改原数据
- view：修改原数据（相当于在原数据上操作）

## pandas入门

#### pandas的数据结构介绍

In [54]:
import numpy as np
from pandas import Series, DataFrame

import pandas as pd

In [30]:
obj = Series([4,7, -5, 3])

obj

0    4
1    7
2   -5
3    3
dtype: int64

In [31]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [32]:
obj.index

RangeIndex(start=0, stop=4, step=1)

In [33]:
# 指定标记索引
obj2 = Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])

obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [34]:
obj2['a']
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

In [35]:
obj2[['c', 'a', 'd']]

c    3
a   -5
d    4
dtype: int64

In [36]:
obj2[obj2>0] # 只是一个副本不修改原来的数据

d    4
b    7
c    3
dtype: int64

In [37]:
obj2

d    4
b    7
a   -5
c    3
dtype: int64

## Series其实可以被看做一个定长的有序字典，一个索引值到数据值的映射

通过Hashtable实现

In [38]:
# 通过Python字典创建Series

sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000}
obj3 = Series(sdata)
obj3 # key 会有序排列

Ohio      35000
Oregon    16000
Texas     71000
Utah       5000
dtype: int64

In [39]:
# 根据 sdata 创建一个对象，修改索引
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = Series(sdata, index=states)

obj4 # 'California 没有匹配项而变成NAN

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [40]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

#### Series在运算中可以自动对齐不同的索引数据

In [41]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

In [42]:
# Series对象本身及其索引都有一个name属性
obj4.name = 'population'
obj4.index.name = 'state'

obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

In [28]:
# 修改索引
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

## DataFrame

创建DataFrame

In [43]:
# dataframe 构建方法有很多，常用的是传入一个
#  由等长列表或Numpy数组组成的字典
# 即传入的是一个字典
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
       'year':[2000, 2001, 2002, 2001, 2002],
       'pop':[1.5, 1.7, 3.6, 2.4, 2.9]}
frame = DataFrame(data)
frame

,pop,state,year
0,1.5,Ohio,2000
1,1.7,Ohio,2001
2,3.6,Ohio,2002
3,2.4,Nevada,2001
4,2.9,Nevada,2002


In [44]:
# 如果在构建是指定了 列的顺序，则会按照指定顺序进行列排序
DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9


In [45]:
# 和Series一样，如果传入的列索引在数据中找不到，就会产生NA值
frame2 = DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                  index = ['one', 'two', 'three', 'four', 'five'])

frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN


In [46]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

In [47]:
# 通过类似字典的方式，可以将DataFrame的列获取为一个Series
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
Name: state, dtype: object

In [49]:
# DataFrame 的行也可以通过位置或名称获取，比如索引ix
frame2.ix['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

In [51]:
# 修改某一列的值
frame2['debt'] = 16.5

frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5


In [55]:
frame2['debt'] = np.arange(5.)

frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0


#### 将列表或数组赋值给某个列时，长度必须跟DataFrame长度匹配。如果赋值是一个Series，就会精确匹配DataFrame的索引，所有空位会被填上NA值。

In [56]:
val = Series([-1.2, -1.5, -1.7], index=['two', 'four', 'six'])

frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,NaN


In [60]:
# 为不存在的列赋值会创建一个新列。 del用于删除列
frame2['eastern'] = frame2.state == 'Ohio'

frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,NaN,False


In [61]:
del frame2['eastern']

frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,NaN


#### 通过索引返回的是视图，对返回的Series做的任何修改都会反映到原DataFrame上。通过Series的copy方法即可显示地复制列。

## 另一种是使用嵌套字典创建DataFrame

#### 这样，外层字典的键作为列，内层键则作为行索引

In [63]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
      'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

frame3 = DataFrame(pop)
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [64]:
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


In [66]:
DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


In [70]:
pdata = {'Ohio': frame3['Ohio'][:-1],
        'Nevada': frame3['Nevada'][:2]}
pdata # frame3 前两行

{'Nevada': 2000    NaN
 2001    2.4
 Name: Nevada, dtype: float64, 'Ohio': 2000    1.5
 2001    1.7
 Name: Ohio, dtype: float64}

In [71]:
DataFrame(pdata)

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7


In [72]:
frame3.index.name = 'year'; frame3.columns.name = 'state'

frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [73]:
# 和Series一样，values属性也会以二维ndarray的形式返回DataFrame中的数据
frame3.values

array([[ nan,  1.5],
       [ 2.4,  1.7],
       [ 2.9,  3.6]])

In [74]:
# 但是如果DataFrame各列的数据类型不同，
# 则数组的数据类型就会选用能兼容所有列的数据类型
# 比如 list
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, nan]], dtype=object)

#### pandas, Series组成有索引对象，用于管理轴标签和其他元数据（比如轴名称等）。

Index对象是不可修改的，immutable

这样才能使Index对象在多个数据结构之间安全共享

In [75]:
obj = Series(range(3), index=['a', 'b', 'c'])
index = obj.index

index

Index(['a', 'b', 'c'], dtype='object')

In [76]:
index[1:]

Index(['b', 'c'], dtype='object')

In [77]:
# 除了长得像数组，Index的功能也类似一个固定大小的集合
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [79]:
'Ohio' in frame3.columns

True

In [80]:
2003 in frame3.index

False

## 基本功能

#### 重新索引

根据新的索引值对Series、DataFrame重排，如果某个索引值当前不存在，就引入缺失值。

In [81]:
obj = Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [82]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [83]:
obj.reindex(['a', 'b', 'c', 'd', 'e'], fill_value=0)

a   -5.3
b    7.2
c    3.6
d    4.5
e    0.0
dtype: float64

In [88]:
# 对于时间序列数据，重新索引时可能需要做一些插值处理，
# method选项可达这样的目的。使用ffill实现前向值填充

obj3 = Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

In [90]:
# 对于DataFrame，reindex可以修改索引（行）、列，或两个都修改。
# 如果仅传入一个序列，则会重新索引行
frame = DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'],
                 columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [91]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [92]:
# 重排索引列，使用columns关键字
states = ['Texas', 'Utah', 'California']

frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [93]:
# 也可以同时对列和行进行重新索引，插值只能按行应用（即轴0）
frame.reindex(index=['a', 'b', 'c', 'd'], method='ffill', columns=states)

,Texas,Utah,California
a,1,NaN,2
b,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [95]:
# 若想使重新索引更加简洁，使用ix标签索引功能
frame.ix[['a', 'b', 'c', 'd'], states]

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


#### 丢弃指定轴上的项

In [97]:
obj = Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
new_obj = obj.drop('c')

new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [99]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

In [100]:
# 对于DataFrame，可以删除任意轴上的索引值
data = DataFrame(np.arange(16).reshape((4, 4)),
                index=['Ohio', 'Colorado', 'Utah', 'New York'],
                columns=['one', 'two', 'three', 'four'])

data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [102]:
data.drop('two', axis=1) #指定轴，如果没有ValueError

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


#### 索引、选取和过滤

但是利用标签的切片运算与普通Python的切片运算不同，末端使包含的，inclusive

In [104]:
obj = Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj['b':'c']

b    1.0
c    2.0
dtype: float64

In [106]:
obj[1:2] # exclusive

b    1.0
dtype: float64

In [108]:
obj['b','c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

In [109]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                index=['Ohio', 'Colorado', 'Utah', 'New York'],
                columns=['one', 'two', 'three', 'four'])

data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [110]:
data[data['three']>5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [116]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


#### 上面的都是对列索引，为了实现对行索引，引入：索引字段ix

In [117]:
data.ix['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int32

In [118]:
data.ix[['Colorado', 'Utah'], [3, 0, 1]]

,four,one,two
Colorado,7,4,5
Utah,11,8,9


In [119]:
data.ix[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [120]:
type(data.ix[0])

pandas.core.series.Series

In [121]:
data.ix['Utah', 'two']

9

In [124]:
data.get_value(index='Utah', col='two')

9

#### 算术运算和数据对齐

In [126]:
s1 = Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])

s2 = Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])

In [127]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [129]:
# 对于DataFrame，对齐操作会同时发生在行和列上
df1 = DataFrame(np.arange(9).reshape(3, 3), columns=list('bcd'),
               index=['Ohio', 'Texas', 'Colorado'])

df2 = DataFrame(np.arange(12).reshape(4, 3), columns=list('bde'),
               index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df1;df2

,b,d,e
Utah,0,1,2
Ohio,3,4,5
Texas,6,7,8
Oregon,9,10,11


In [130]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


#### 在算术方法中填充值

In [131]:
df1 = DataFrame(np.arange(12).reshape(3, 4), columns=list('abcd'))
df2 = DataFrame(np.arange(20).reshape(4, 5), columns=list('abcde'))
df1

,a,b,c,d
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [133]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,11.0,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [134]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [135]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0,1,2,3,0
1,4,5,6,7,0
2,8,9,10,11,0


#### DataFrame和Series之间的运算

先看一个numpy二维数组与其某行之间的差

In [136]:
arr = np.arange(12.).reshape((3, 4))
arr

array([[  0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.]])

In [137]:
arr - arr[0]

array([[ 0.,  0.,  0.,  0.],
       [ 4.,  4.,  4.,  4.],
       [ 8.,  8.,  8.,  8.]])

In [138]:
frame = DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                 index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.ix[0]
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [139]:
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

In [140]:
# 默认情况下，DataFrame和Series之间的算术运算会将series的索引匹配到DataFrame的列，
# 然后沿着行一直向下广播

frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [144]:
series2 = Series(range(3), index=['b', 'e', 'f'])
series2

b    0
e    1
f    2
dtype: int32

In [142]:
# 如果某个索引值在DataFrame中找不到，则参与运算的两个对象就会被重新索引
# 以形成并集
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


In [145]:
# 如果希望匹配行在列上广播，则必须使用算术运算方法
series3 = frame['d']
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [146]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [ ]:
frame.sub(series3, axi)